In [17]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder,StandardScaler,RobustScaler,OneHotEncoder

def load_application_train():
    data = pd.read_csv("application_train.csv")
    return data 

def load_titanic():
    data = pd.read_csv("titanic.csv")
    return data 

df = load_application_train()
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
df["AMT_REQ_CREDIT_BUREAU_HOUR"].value_counts()

AMT_REQ_CREDIT_BUREAU_HOUR
0.0    264366
1.0      1560
2.0        56
3.0         9
4.0         1
Name: count, dtype: int64

In [19]:
def grab_col_names(df,cat_th=10,car_th=20):
    num_cols = [col for col in df.columns if np.issubdtype(df[col].dtype, np.number)]
    cat_cols = [col for col in df.columns if col not in num_cols]
    num_but_cat =  [col for col in num_cols if df[col].nunique()<cat_th]
    cat_but_car = [col for col in cat_cols if df[col].nunique()>car_th]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]
    num_cols = [col for col in num_cols if col not in num_but_cat]
    print(f"cat_cols = {len(cat_cols)}")
    print(f"num_cols ={len(num_cols)}")
    print(f"cat_but_car = {len(cat_but_car)}")
    print(f"num_but_cat={len(num_but_cat)}")
        
    return cat_cols, num_cols,cat_but_car

cat_cols, num_cols,cat_but_car=grab_col_names(df)
cat_cols

cat_cols = 54
num_cols =67
cat_but_car = 1
num_but_cat=39


['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'FONDKAPREMONT_MODE',
 'HOUSETYPE_MODE',
 'WALLSMATERIAL_MODE',
 'EMERGENCYSTATE_MODE',
 'TARGET',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'DEF_60_CNT_SOCIAL_CIRCLE',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOC

In [20]:
def rare_analysis(df,target,cat_cols):
    for col in cat_cols:
        print(col,":",len(df[col].value_counts()))
        df_ratio = (df[col].value_counts() /len(df)).sort_values(ascending=False)
        df_count = df[col].value_counts().loc[df_ratio.index]
        df_target_mean = df.groupby(col)[target].mean().loc[df_ratio.index]
        print(pd.DataFrame({"Ratio": df_ratio,
              "Count": df_count,
              "Target_Mean": df_target_mean}))
        print("-------------------------------------------------------")
        
rare_analysis(df,"TARGET",cat_cols)

NAME_CONTRACT_TYPE : 2
                       Ratio   Count  Target_Mean
NAME_CONTRACT_TYPE                               
Cash loans          0.904787  278232     0.083459
Revolving loans     0.095213   29279     0.054783
-------------------------------------------------------
CODE_GENDER : 3
                Ratio   Count  Target_Mean
CODE_GENDER                               
F            0.658344  202448     0.069993
M            0.341643  105059     0.101419
XNA          0.000013       4     0.000000
-------------------------------------------------------
FLAG_OWN_CAR : 2
                 Ratio   Count  Target_Mean
FLAG_OWN_CAR                               
N             0.659892  202924     0.085002
Y             0.340108  104587     0.072437
-------------------------------------------------------
FLAG_OWN_REALTY : 2
                    Ratio   Count  Target_Mean
FLAG_OWN_REALTY                               
Y                0.693673  213312     0.079616
N                0.30632

In [12]:
df["AMT_REQ_CREDIT_BUREAU_WEEK"].value_counts().loc[4.0] / len(df)

0.00011056515051494093

In [16]:
df.loc[df["AMT_REQ_CREDIT_BUREAU_WEEK"]==4,["AMT_REQ_CREDIT_BUREAU_WEEK","TARGET"]]["TARGET"].mean()

0.11764705882352941

In [26]:
df2 = load_titanic()

In [ ]:
def merge_c(df2, cols, target_col, threshold):
    temp_df = df2.copy()
    for col in cols:
        unique_categories = temp_df[col].value_counts().index.tolist()

        # Eğer yalnızca bir tane kategori varsa, birleştirme işlemine gerek yoktur
        if len(unique_categories) > 1:
            for category in unique_categories:
                category_freq = df[col].value_counts().loc[category] / len(df)

                # Eğer kategori frekansı eşik değerinin altındaysa, birleştirme işlemi gerçekleştirilir
                if category_freq < threshold:
                    # Kendi kategorisini ve hedef ortalamalarını al
                    own_target_mean = temp_df.loc[temp_df[col] == category,[col,target_col]][target_col].mean()
                    
                    # Kendi dışındaki kategorilerin hedef ortalamalarını al
                    other_categories = temp_df.loc[temp_df[col] != category, [col, target_col]]
                    other_target_means = other_categories.groupby(col)[target_col].mean()
                    
                    # Kendi hedef ortalamasına en yakın olan kategoriyi bul
                    closest_category = (other_target_means - own_target_mean).abs().idxmin()
                    
                    # Kendi kategorisini, en yakın olan kategori ile değiştir
                    temp_df[col] = temp_df[col].replace(category, closest_category)
            
    return temp_df

df3 = merge_c(df, cat_cols, "TARGET", 0.03)

In [23]:
rare_analysis(df3,"TARGET",cat_cols)

NAME_CONTRACT_TYPE : 2
                       Ratio   Count  Target_Mean
NAME_CONTRACT_TYPE                               
Cash loans          0.904787  278232     0.083459
Revolving loans     0.095213   29279     0.054783
-------------------------------------------------------
CODE_GENDER : 2
                Ratio   Count  Target_Mean
CODE_GENDER                               
F            0.658357  202452     0.069992
M            0.341643  105059     0.101419
-------------------------------------------------------
FLAG_OWN_CAR : 2
                 Ratio   Count  Target_Mean
FLAG_OWN_CAR                               
N             0.659892  202924     0.085002
Y             0.340108  104587     0.072437
-------------------------------------------------------
FLAG_OWN_REALTY : 2
                    Ratio   Count  Target_Mean
FLAG_OWN_REALTY                               
Y                0.693673  213312     0.079616
N                0.306327   94199     0.083249
--------------------

In [21]:
rare_analysis(df,"TARGET",cat_cols)

NAME_CONTRACT_TYPE : 2
                       Ratio   Count  Target_Mean
NAME_CONTRACT_TYPE                               
Cash loans          0.904787  278232     0.083459
Revolving loans     0.095213   29279     0.054783
-------------------------------------------------------
CODE_GENDER : 3
                Ratio   Count  Target_Mean
CODE_GENDER                               
F            0.658344  202448     0.069993
M            0.341643  105059     0.101419
XNA          0.000013       4     0.000000
-------------------------------------------------------
FLAG_OWN_CAR : 2
                 Ratio   Count  Target_Mean
FLAG_OWN_CAR                               
N             0.659892  202924     0.085002
Y             0.340108  104587     0.072437
-------------------------------------------------------
FLAG_OWN_REALTY : 2
                    Ratio   Count  Target_Mean
FLAG_OWN_REALTY                               
Y                0.693673  213312     0.079616
N                0.30632

In [ ]:
def merge_c(df2, cols, target_col, threshold):
    temp_df = df2.copy()
    for col in cols:
        unique_categories = temp_df[col].value_counts().index.tolist()

        # Eğer yalnızca bir tane kategori varsa, birleştirme işlemine gerek yoktur
        if len(unique_categories) > 1:
            for category in unique_categories:
                category_freq = temp_df[temp_df[col] == category].shape[0] / len(temp_df)

                # Eğer kategori frekansı eşik değerinin altındaysa, birleştirme işlemi gerçekleştirilir
                if category_freq < threshold:
                    # Kendi kategorisini ve hedef ortalamalarını al
                    own_target_mean = temp_df[temp_df[col] == category][target_col].mean()
                    
                    # Kendi dışındaki kategorilerin hedef ortalamalarını al
                    other_categories = temp_df.loc[temp_df[col] != category, [col, target_col]]
                    other_target_means = other_categories.groupby(col)[target_col].mean()
                    
                    # Kendi hedef ortalamasına en yakın olan kategoriyi bul
                    closest_category = (other_target_means - own_target_mean).abs().idxmin()
                    
                    # Kendi kategorisini, en yakın olan kategori ile değiştir
                    temp_df[col] = temp_df[col].replace(category, closest_category)
            
    return temp_df

a=merge_c(df2,["Embarked"],"Survived",0.05)
a

In [31]:
a.Embarked.unique()

array(['S', 'C', 'Q', nan], dtype=object)

In [40]:
for col in a.Embarked.unique():
    print(a[a["Embarked"] == col]["Survived"].mean())

0.33695652173913043
0.5535714285714286
0.38961038961038963
nan


In [44]:
df4 = df.copy()

In [46]:
df4[cat_cols]

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,...,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK
0,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,...,0,0,0,0,0,0,0,0.0,0.0,0.0
1,Cash loans,F,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,...,0,0,0,0,0,0,0,0.0,0.0,0.0
2,Revolving loans,M,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,...,0,0,0,0,0,0,0,0.0,0.0,0.0
3,Cash loans,F,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,...,0,0,0,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,Cash loans,M,N,N,Unaccompanied,Working,Secondary / secondary special,Separated,With parents,Sales staff,...,0,0,0,0,0,0,0,NaN,NaN,NaN
307507,Cash loans,F,N,Y,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
307508,Cash loans,F,N,Y,Unaccompanied,Working,Higher education,Separated,House / apartment,Managers,...,0,0,0,0,0,0,0,1.0,0.0,0.0
307509,Cash loans,F,N,Y,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,Laborers,...,0,0,0,0,0,0,0,0.0,0.0,0.0


In [49]:
def rare_enc(df,cat_cols,th=0.03):
    for col in cat_cols:
        cat_perc = df[col].value_counts() / len(df)
        cat_list = list(cat_perc[cat_perc<th].index)
        
        if len(cat_list)>=1:
            df[col] = df[col].replace(cat_list,"rare")
            
            
rare_enc(df,cat_cols,th=0.03)

In [50]:
rare_analysis(df,"TARGET",cat_cols)

NAME_CONTRACT_TYPE : 2
                       Ratio   Count  Target_Mean
NAME_CONTRACT_TYPE                               
Cash loans          0.904787  278232     0.083459
Revolving loans     0.095213   29279     0.054783
-------------------------------------------------------
CODE_GENDER : 3
                Ratio   Count  Target_Mean
CODE_GENDER                               
F            0.658344  202448     0.069993
M            0.341643  105059     0.101419
rare         0.000013       4     0.000000
-------------------------------------------------------
FLAG_OWN_CAR : 2
                 Ratio   Count  Target_Mean
FLAG_OWN_CAR                               
N             0.659892  202924     0.085002
Y             0.340108  104587     0.072437
-------------------------------------------------------
FLAG_OWN_REALTY : 2
                    Ratio   Count  Target_Mean
FLAG_OWN_REALTY                               
Y                0.693673  213312     0.079616
N                0.30632

In [51]:
na_cols = [col for col in df.columns if df[col].isnull().sum()>0]
na_cols

['AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'OWN_CAR_AGE',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MODE',
 'COMMONAREA_MODE',
 'ELEVATORS_MODE',
 'ENTRANCES_MODE',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MODE',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MODE',
 'APARTMENTS_MEDI',
 'BASEMENTAREA_MEDI',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'YEARS_BUILD_MEDI',
 'COMMONAREA_MEDI',
 'ELEVATORS_MEDI',
 'ENTRANCES_MEDI',
 'FLOORSMAX_MEDI',
 'FLOORSMIN_MEDI',
 'LANDAREA_MEDI',
 'LIVINGAPARTMENTS_MEDI',

In [52]:
def missing_vs_target(df,target):
    na_cols = [col for col in df.columns if df[col].isnull().sum()>0]
    temp_df=df.copy()
    for col in na_cols:
        temp_df[col+"_NA_Flag"] = np.where(temp_df[col].isnull(),1,0)
    na_flags=[col for col in temp_df.columns if "_NA_" in col]
    for col in na_flags:
        print(pd.DataFrame({"Target_Mean":temp_df.groupby(col)[target].mean(),
                            "Count":temp_df.groupby(col)[target].count()}))
        print("-----------------------------------------------")

missing_vs_target(df,"TARGET")

                     Target_Mean   Count
AMT_ANNUITY_NA_Flag                     
0                       0.080732  307499
1                       0.000000      12
-----------------------------------------------
                         Target_Mean   Count
AMT_GOODS_PRICE_NA_Flag                     
0                           0.080734  307233
1                           0.075540     278
-----------------------------------------------
                         Target_Mean   Count
NAME_TYPE_SUITE_NA_Flag                     
0                           0.080841  306219
1                           0.054180    1292
-----------------------------------------------
                     Target_Mean   Count
OWN_CAR_AGE_NA_Flag                     
0                       0.072441  104582
1                       0.085000  202929
-----------------------------------------------
                         Target_Mean   Count
OCCUPATION_TYPE_NA_Flag                     
0                           0.